In [ ]:
"""
Purpose: To check the progress of version 4
pods running

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-19 04:02:10,226 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,229 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,230 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,234 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-19 04:02:10,235 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:10,247 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-19 04:02:10,692 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:10,706 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,707 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,708 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,709 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,710 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,711 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,712 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:11,198 - settings - Setting enable_python_native_blobs to True


In [3]:
import datajoint as dj
version = 88
nucleus_table_ids = du.nucleus_info_table(version = 88) & "segment_id>0"
minnie, schema = du.configure_minnie_vm()


INFO - 2021-05-19 04:02:11,372 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:11,373 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:11,374 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:11,378 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:11,913 - settings - Setting enable_python_native_blobs to True


# The list of segments

In [4]:
minnie.NucleiSegmentsRun4()

segment_id segment id for those to be decimated
864691131559524855
864691131658969700
864691131705161142
864691131771477291
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131906133483


# Decimation

In [5]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
curr_table

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decimation,00377f5b109485e869a5defb2f6b9c46,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427072,2021-05-17 16:00:35
__decimation,0577fe53aed3f61fa48c14cace77c0d5,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.157,at-node30,1,427051,2021-05-17 16:00:17
__decimation,098e0be9fd9ace0ef11c71e33c872874,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.140,at-node13,1,427070,2021-05-17 15:54:37
__decimation,0b32188cc97c146c758bf0868d25e350,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.155,at-node28,1,427093,2021-05-17 16:00:28
__decimation,0b9a0dc68db9c46ed591d4d6fb9f428d,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.66.4.27,at-compute005,1,427063,2021-05-17 16:00:19
__decimation,17c785a4b4690cb9745c874d38ae6638,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.164,at-node37,1,427054,2021-05-17 16:00:19
__decimation,18cd69c4da59d8e5637c21fb14470057,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.133,at-node6,1,427056,2021-05-17 15:50:59
__decimation,1cf6ccb3ad6fd43fafb08cca317df795,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427078,2021-05-17 16:00:25
__decimation,1ee98a5dabebf5e40832306e5e0cf5cb,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427082,2021-05-17 16:00:21
__decimation,20007fb0b96ab55246a54530e15b4f19,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.159,at-node32,1,427089,2021-05-17 16:00:21


# Soma Extraction

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table

In [ ]:
len(minnie.BaylorSegmentCentroid()) - 102595

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (
                                                                  du.proofreading_segment_id_restriction()
                                                                 ))
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.BaylorSegmentCentroid())

# 4) Decomposition

In [ ]:
key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")
                  & du.proofreading_segment_id_restriction())
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.Decomposition())

In [18]:
len(minnie.Decomposition()) - 77723

INFO - 2021-05-19 15:47:29,974 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:29,982 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:29,983 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:29,983 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:29,984 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:29,984 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:29,989 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:47:30,756 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:47:31,253 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:31,254 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:31,262 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:31,266 - settings - Setting enable_pytho

53

In [19]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
curr_table
#du.restrict_jobs_table_by_error_substring(curr_table,"SystemExit: SIGTERM received").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,00209cbad4844284a34c87b29daa0990,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,435227,2021-05-19 10:11:26
__decomposition,016e1e5be5a73ed15fc65f5c8c968aa0,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.159,at-node32,1,435577,2021-05-19 09:51:21
__decomposition,0208b9a0f2826d2c26b033e4c3420751,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.142,at-node15,1,435390,2021-05-19 09:38:16
__decomposition,027a3715e3aa47b940b69560ccb1388d,error,=BLOB=,AttributeError: 'list' object has no attribute 'reshape',=BLOB=,celiib@10.66.4.27,at-compute005,1,435185,2021-05-19 09:04:27
__decomposition,02bada258faa726f300eb949b292f5a6,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.161,at-node34,1,435319,2021-05-19 09:23:16
__decomposition,0573816bcc8ef345cd6ad9cd0380724f,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.177,at-node50,1,435212,2021-05-19 10:11:25
__decomposition,05c15c053e510d1782fc019bcc0d1b79,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.142,at-node15,1,435613,2021-05-19 09:50:30
__decomposition,061b32ba60d99ba66d7efe670f50156e,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,1,435567,2021-05-19 09:43:49
__decomposition,07c3951dd3214dd9a3d0d30e6fc5dd88,error,=BLOB=,"Exception: neuron 208816 did not fix the manifold edges (meshlab script failed) with output:QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root' Current Plugins Dir is: /meshlab/src/distrib/plugins Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Opening a file with extention off FilterScript Reading filter with name Remove Duplicate Vertices Reading filter with name Smooths normals on a point sets Reading Param with name K : RichInt Reading Param with name useDist : RichBool Reading filter with name Surface Reconstruction: Screened Poisson Reading filter with name Delete Current Mesh Loading Plugins: Total 104 filtering actions Total 1 io plugins Mesh /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/neuron_208816.off loaded has 8141 vn 16082 fn output mesh /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/neuron_208816_poisson.off Apply FilterScript: '/notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/poisson_23675.mls' Starting Script of 4 actionsfilter: Remove Duplicate Vertices no additional memory available!!! memory required: 388368 LOG: 2 Removed 0 duplicated vertices Removed 0 duplicated vertices filter: Smooths normals on a point sets no additional memory available!!! memory required: 388368 Removed 0 duplicated vertices filter: Surface Reconstruction: Screened Poisson no additional memory available!!! memory required: 388368 Removed 0 duplicated vertices Problem with filter: Surface Reconstruction: Screened Poisson Failed to apply script file /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/poisson_23675.mls",=BLOB=,celiib@10.66.4.27,at-compute005,1,435105,2021-05-19 08:25:43
__decomposition,07c59b759f50a6dfd723f1ed1b289da5,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,1,435488,2021-05-19 09:25:47


# DecompositionAxon

In [ ]:
"""
NOTE: quit some jobs at 11:20 so could erase those from the jobs 
schema

"""

In [ ]:
"""
Reminder have to eliminate the jobs that were ended prior to running
the new batch

Already did using: 
# curr_table = (minnie.schema.jobs & "table_name='__decomposition_axon'")
# (curr_table & "timestamp<'2021-05-17 14:00:03'").delete()

"""

In [46]:
len(minnie.DecompositionAxon())

INFO - 2021-05-21 14:12:45,535 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:12:45,536 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:12:45,537 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:12:45,538 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:12:45,539 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:12:45,540 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:12:45,559 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:12:46,608 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:12:47,531 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:12:47,532 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:12:47,533 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:12:47,535 - settings - Setting enable_pytho

40676

In [ ]:
len(minnie.Decomposition()) - 75967

In [ ]:
minnie.Decomposition()

In [25]:
key_source = minnie.Decomposition() & du.proofreading_segment_id_restriction()
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.DecompositionAxon())

segment_id segment id unique within each Segmentation
864691135576941829
864691135538047346
864691135373445576
864691135341191601
864691135446618982
864691135918326704
864691135274346981
864691135570705133
864691135162270253
864691135845412852


# Getting the Synapses Filtered Away

In [ ]:
return_error_table_entries

keys_to_write_without_version_errors

keys_to_write_errors

AutoProofreadSynapseErrors_keys